In [62]:
#%qtconsole --style monokai --ConsoleWidget.font_size=11

In [63]:
__author__ = 'sharmaam'

import pandas as pd
import numpy as np
import os
import sys
import glob
from sklearn import cross_validation, ensemble, preprocessing, metrics
from sklearn.grid_search import GridSearchCV
import collections
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import xgboost as xgb


sys.path.append('../../common/') 
import kappa

pd.set_option('display.max_columns', None)

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

# Prints the feature importance sorted descending for a model
def featureImp(model, df, n=10):
    index = np.argsort(model.feature_importances_)[::-1]
    print df.columns[index].values[:n] + ':' + model.feature_importances_[index].astype(str)[:n]


# Percentage Root Mean Squared Error (RMSPE)
def rmspe(pred, actual):
    sse = np.sum(pow((pred-actual)/pred, 2))
    sseMean = sse / len(pred)
    rmspe = np.sqrt(sseMean)
    return rmspe


# Print Grid Search results
def getBestModel(model):
    print("Best score: %0.3f" % model.best_score_)
    print("Best parameters set:")
    best_parameters = model.best_estimator_.get_params()
    for param_name in sorted(param_grid.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
    return model.best_estimator_

# Returns true for encoded columns
def isEncodedCol(df, col):
    encodedCols = []
    encode = (df[col].dtype != 'int64') & (df[col].dtype != 'float64')
    encode = encode | (col in encodedCols)
    return encode

# Encode columns to categorical values
def encodeCols(df):
    print "Encoding cols..."
    # Encode categorical cols
    for col in df.columns:
        if (isEncodedCol(df, col)):
            lbl = preprocessing.LabelEncoder()
            lbl.fit(df[col])
            df[col] = lbl.transform(df[col])
    return df


def preProcess(df):
#     df['Month'] = df.Date.dt.month.astype(int)
#     df['Day'] = df.Date.dt.day.astype(int)
#     df['Year'] = df.Date.dt.year.astype(int)

#     grpByCust = df.groupby(["Store", "Month"]).mean()['Customers']
#     grpByCust.name = 'Avg_Cust'
#     df = df.join(grpByCust, on=['Store', 'Month'])

#     stores = pd.read_csv('../input/store.csv')
#     df = df.merge(stores, on='Store')    
    
    catCols = ["Product_Info_1", "Product_Info_2", "Product_Info_3", "Product_Info_5", "Product_Info_6", 
                "Product_Info_7", "Employment_Info_2", "Employment_Info_3", "Employment_Info_5", "InsuredInfo_1", 
                  "InsuredInfo_2", "InsuredInfo_3", "InsuredInfo_4", "InsuredInfo_5", "InsuredInfo_6", "InsuredInfo_7", 
                   "Insurance_History_1", "Insurance_History_2", "Insurance_History_3", "Insurance_History_4", 
                   "Insurance_History_7", "Insurance_History_8", "Insurance_History_9", "Family_Hist_1", 
                   "Medical_History_2", "Medical_History_3", "Medical_History_4", "Medical_History_5", 
                   "Medical_History_6", "Medical_History_7", "Medical_History_8", "Medical_History_9", 
                   "Medical_History_10", "Medical_History_11", "Medical_History_12", "Medical_History_13", 
                   "Medical_History_14", "Medical_History_16", "Medical_History_17", "Medical_History_18", 
                   "Medical_History_19", "Medical_History_20", "Medical_History_21", "Medical_History_22", 
                   "Medical_History_23", "Medical_History_25", "Medical_History_26", "Medical_History_27", 
                   "Medical_History_28", "Medical_History_29", "Medical_History_30", "Medical_History_31", 
                   "Medical_History_33", "Medical_History_34", "Medical_History_35", "Medical_History_36", 
                   "Medical_History_37", "Medical_History_38", "Medical_History_39", "Medical_History_40", "Medical_History_41"]
    
    for catCol in catCols:
        dummyDF = pd.get_dummies(df[catCol], prefix=catCol)
        df = pd.concat([df, dummyDF], axis=1)
        
    # Remove columns not being handled currently
#     df = df.drop(['Customers'], axis=1)

    # Impute missing values
    for impCol in df.columns:
        if (df[impCol].dtype == 'int64') or (df[impCol].dtype == 'float64'):
            #imputedVal = df[impCol].value_counts().index[0]
            #df[impCol] = df[impCol].fillna(imputedVal)
            df[impCol] = df[impCol].fillna(0)
        elif df[impCol].dtype == 'object':
            #imputedVal = df[impCol].value_counts().index[0]
            #df[impCol] = df[impCol].fillna(imputedVal)
            df[impCol] = df[impCol].fillna('NAValue')

    '''
    print "Cleaning too many categories..."
    ### Clean variables with too many categories
    for col in df.columns:
        if (df[col].dtype != 'int64') & (df[col].dtype != 'float64'):
            top30 = np.array(collections.Counter(df[col]).most_common(60))[:, 0]
            notInTop30 = [(not x in top30) for x in df[col]]
            df.ix[notInTop30, col] = 'rareValue'

    '''
    return df



In [64]:
forSubmission = False
gbr = False

# load trainAlling and test datasets
trainAll = pd.read_csv('../input/train.csv')

# Take a random split for temp training purpose
trainRows, testRows = train_test_split(trainAll.index, test_size = 0.2)

trainAll = preProcess(trainAll)
trainAll = encodeCols(trainAll)

train = trainAll.iloc[trainRows]
test = trainAll.iloc[testRows]

Encoding cols...


In [65]:
train.head()

,Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Family_Hist_2,Family_Hist_3,Family_Hist_4,Family_Hist_5,Medical_History_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_10,Medical_History_11,Medical_History_12,Medical_History_13,Medical_History_14,Medical_History_15,Medical_History_16,Medical_History_17,Medical_History_18,Medical_History_19,Medical_History_20,Medical_History_21,Medical_History_22,Medical_History_23,Medical_History_24,Medical_History_25,Medical_History_26,Medical_History_27,Medical_History_28,Medical_History_29,Medical_History_30,Medical_History_31,Medical_History_32,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41,Medical_Keyword_1,Medical_Keyword_2,Medical_Keyword_3,Medical_Keyword_4,Medical_Keyword_5,Medical_Keyword_6,Medical_Keyword_7,Medical_Keyword_8,Medical_Keyword_9,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response,Product_Info_1_1,Product_Info_1_2,Product_Info_2_A1,Product_Info_2_A2,Product_Info_2_A3,Product_Info_2_A4,Product_Info_2_A5,Product_Info_2_A6,Product_Info_2_A7,Product_Info_2_A8,Product_Info_2_B1,Product_Info_2_B2,Product_Info_2_C1,Product_Info_2_C2,Product_Info_2_C3,Product_Info_2_C4,Product_Info_2_D1,Product_Info_2_D2,Product_Info_2_D3,Product_Info_2_D4,Product_Info_2_E1,Product_Info_3_1,Product_Info_3_2,Product_Info_3_3,Product_Info_3_4,Product_Info_3_5,Product_Info_3_6,Product_Info_3_8,Product_Info_3_9,Product_Info_3_10,Product_Info_3_11,Product_Info_3_12,Product_Info_3_13,Product_Info_3_15,Product_Info_3_16,Product_Info_3_17,Product_Info_3_18,Product_Info_3_19,Product_Info_3_20,Product_Info_3_21,Product_Info_3_22,Product_Info_3_23,Product_Info_3_24,Product_Info_3_26,Product_Info_3_27,Product_Info_3_28,Product_Info_3_29,Product_Info_3_30,Product_Info_3_31,Product_Info_3_32,Product_Info_3_33,Product_Info_3_34,Product_Info_3_36,Product_Info_3_37,Product_Info_3_38,Product_Info_5_2,Product_Info_5_3,Product_Info_6_1,Product_Info_6_3,Product_Info_7_1,Product_Info_7_2,Product_Info_7_3,Employment_Info_2_1,Employment_Info_2_2,Employment_Info_2_3,Employment_Info_2_4,Employment_Info_2_5,Employment_Info_2_6,Employment_Info_2_7,Employment_Info_2_9,Employment_Info_2_10,Employment_Info_2_11,Employment_Info_2_12,Employment_Info_2_13,Employment_Info_2_14,Employment_Info_2_15,Employment_Info_2_16,Employment_Info_2_17,Employment_Info_2_18,Employment_Info_2_19,Employment_Info_2_20,Employment_Info_2_21,Employment_Info_2_22,Employment_Info_2_23,Employment_Info_2_25,Employment_Info_2_26,Employment_Info_2_27,Employment_Info_2_28,Employment_Info_2_29,Employment_Info_2_30,Employment_Info_2_31,Employment_Info_2_32,Employment_Info_2_33,Employment_In

In [66]:
trainY = train.Response-1
trainX = train.drop(["Response"], axis=1)
train.shape

(47504, 1060)

In [ ]:
model = ensemble.RandomForestClassifier(n_estimators=500, oob_score=True, max_features=None, max_depth=None).fit(trainX, trainY)

# params = {}
# params["objective"] = "multi:softmax"
# params["eta"] = 0.02
# params["min_child_weight"] = 6
# params["subsample"] = 0.7
# params["colsample_bytree"] = 0.6
# params["scale_pos_weight"] = 0.8
# params["silent"] = 1
# params["max_depth"] = 20
# params["max_delta_step"]=2
# params["num_class"]=trainY.unique().shape[0]

# plst = list(params.items())

# xgtrain = xgb.DMatrix(trainX, label=trainY)
# xgtest = xgb.DMatrix(test)


# scale=10
# print("-----------------------------------------------------------")
# print("Running with a scale factor: ", scale)
# print("-----------------------------------------------------------")

# print('15000')


# num_rounds = 1500/scale
# model = xgb.train(plst, xgtrain, num_rounds)
# preds = model.predict(xgtest)

# print "...Completed!"

# Grid based turning
# from sklearn import linear_model
# model = linear_model.LinearRegression(fit_intercept=True, normalize=True, copy_X=True).fit(trainX, trainY)
# Alternative model
# rfModel = ensemble.RandomForestRegressor(criterion='mse')
# param_grid = dict(max_features=[40,50,60,70,80,90])
# model = GridSearchCV(rfModel, param_grid=param_grid, scoring=rmsle_scorer, verbose=10).fit(trainX, trainY)
# model = getBestModel(model)

In [68]:
#preds = model.predict(test.ix[:,:-1])

print "Kappa Score: ", kappa.quadratic_weighted_kappa(test.Response, preds, 
                                                      np.min(test.Response), np.max(test.Response))

Kappa Score:  0.034150630944
